In [2]:
################## IMPORT ######################
import json
from datetime import datetime
from functools import partial, wraps

import fire
import jax
import jax.numpy as jnp
import numpy as np
from jax import jit, random, value_and_grad, vmap
# from jax.experimental import optimizers
from jax.example_libraries import optimizers
from jax_md import space
from shadow.plot import *
from sklearn.metrics import r2_score
from psystems.nsprings import (chain, edge_order, get_connections,
                               get_fully_connected_senders_and_receivers,
                               get_fully_edge_order)
# from statistics import mode
# from sympy import LM
# from torch import batch_norm_gather_stats_with_counts
import sys
MAINPATH = ".."  # nopep8
sys.path.append(MAINPATH)  # nopep8
import jraph
import src
from jax.config import config
# from src import fgn, lnn
from src.graph import *
# from src.lnn import acceleration, accelerationFull, accelerationTV
from src.md import *
from src.models import MSE, initialize_mlp, GaussianNLL, initialize_mlp_gamma, forward_pass_gamma
from src.nve import NVEStates, nve, BrownianStates
from src.utils import *

# config.update("jax_enable_x64", True)
# config.update("jax_debug_nans", True)
# jax.config.update('jax_platform_name', 'gpu')

def namestr(obj, namespace):
    return [name for name in namespace if namespace[name] is obj]


def pprint(*args, namespace=globals()):
    for arg in args:
        print(f"{namestr(arg, namespace)[0]}: {arg}")

f32 = jnp.float32
f64 = jnp.float64

In [3]:
N = 5  # number of particles
dim = 2  # dimensions
runs = 1
kT = 1 #1.380649e-23*T  # boltzmann constant*temperature
# spring_constant = 1.0
# length_constant = 1.0
# nconfig=100
seed=42
dt = 1.0e-3 # time step*stride 
lr=1e-4
batch_size=20
epochs = 10000
# node_type = jnp.array([0,0,0,0,0])
masses = jnp.ones(N)
species = jnp.zeros(N, dtype=int).reshape(-1,1)
# gamma = jnp.ones(jnp.unique(species).shape)  # damping constant

rname=True
withdata = None

print("Configs: ")
pprint(N, epochs, seed, rname, dt, lr, batch_size, namespace=locals())

randfilename = datetime.now().strftime("%m-%d-%Y_%H-%M-%S")

PSYS = f"a-{N}-non-linear-Spring-data-brownian_EM"
TAG = f"3BFGN"
out_dir = f"../results"

def _filename(name, tag=TAG):
    rstring = randfilename if (rname and (tag != "data")) else (
        "0" if (tag == "data") or (withdata == None) else f"0_{withdata}")
    filename_prefix = f"{out_dir}/{PSYS}-{tag}/{rstring}/"
    file = f"{filename_prefix}/{name}"
    os.makedirs(os.path.dirname(file), exist_ok=True)
    filename = f"{filename_prefix}/{name}".replace("//", "/")
    print("===", filename, "===")
    return filename

def displacement(a, b):
    return a - b

def shift(R, dR):
    return R+dR

def OUT(f):
    @wraps(f)
    def func(file, *args, tag=TAG, **kwargs):
        return f(_filename(file, tag=tag), *args, **kwargs)
    return func

loadmodel = OUT(src.models.loadmodel)
savemodel = OUT(src.models.savemodel)

loadfile = OUT(src.io.loadfile)
savefile = OUT(src.io.savefile)
save_ovito = OUT(src.io.save_ovito)


Configs: 
N: 5
epochs: 10000
seed: 42
rname: True
dt: 0.001
lr: 0.0001
batch_size: 20


In [4]:
################################################
################## CONFIG ######################
################################################
np.random.seed(seed)
key = random.PRNGKey(seed)

try:
    dataset_states = loadfile(f"model_states_brownian.pkl", tag="data")[0]
except:
    raise Exception("Generate dataset first.")

model_states = dataset_states[0]

print(f"Total number of data points: {len(dataset_states)}x{model_states.position.shape[0]}")

Rs = States_Brow().fromlist(dataset_states).get_array()

Rs_in = Rs[:,:99,:,:]
Rs_out = Rs[:,1:100,:,:]


=== ../results/a-5-non-linear-Spring-data-brownian_EM-data/0/model_states_brownian.pkl ===
Total number of data points: 100x100


In [5]:
from functools import partial

import haiku as hk
import jax
import jax.numpy as jnp
import numpy as np
from jax import grad, jit, lax, random
from jax_md.nn import GraphNetEncoder
from jraph import GraphMapFeatures, GraphNetwork, GraphsTuple

from src.models import SquarePlus, forward_pass, initialize_mlp


class GraphEncodeNet():
    def __init__(self, N, embedding_fn, model_fn, final_fn):
        self.N = N
        self._encoder = GraphMapFeatures(
            embedding_fn('EdgeEncoder'),
            embedding_fn('NodeEncoder'),
            embedding_fn('GlobalEncoder'))
        self._propagation_network = GraphNetwork(
            model_fn('EdgeFunction'),
            model_fn('NodeFunction'),
            model_fn('GlobalFunction'), aggregate_edges_for_globals_fn=lambda *x: jnp.array([0.0]))
        self._final = GraphNetwork(
            final_fn('EdgeFunction'),
            final_fn('NodeFunction'),
            final_fn('GlobalFunction'), aggregate_edges_for_globals_fn=lambda *x: jnp.array([0.0]))

    def __call__(self, graph):
        output = self._encoder(graph)
        for _ in range(self.N):
            output = self._propagation_network(output)
        output = self._final(output)
        return output


def cal(params, graph, mpass=1):
    ee_params = params["ee_params"]
    ne_params = params["ne_params"]
    e_params = params["e_params"]
    n_params = params["n_params"]
    g_params = params["g_params"]

    def node_em(nodes):
        out = jnp.hstack([v for k, v in nodes.items()])
        def fn(out):
            return forward_pass(ne_params, out, activation_fn=SquarePlus)
        out = jax.vmap(fn)(out)
        return {"embed": out}
    
    def edge_em(edges):
        out = edges["dij"]
        out = jax.vmap(lambda p, x: forward_pass(p, x.reshape(-1)),
                       in_axes=(None, 0))(ee_params, out)
        return {"embed": out}

    embedding = {
        "EdgeEncoder": edge_em,
        "NodeEncoder": node_em,
        "GlobalEncoder": None,
    }
    
    def embedding_fn(arg): return embedding[arg]

    def edge_fn(edges, sent_attributes, received_attributes, global_):
        out = jnp.hstack([edges["embed"], sent_attributes["embed"],
                          received_attributes["embed"]])
        out = jax.vmap(forward_pass, in_axes=(None, 0))(e_params, out)
        return {"embed": out}

    def node_fn(nodes, sent_attributes, received_attributes, global_):
        out = jnp.hstack([nodes["embed"], sent_attributes["embed"],
                          received_attributes["embed"]])
        out = jax.vmap(forward_pass, in_axes=(None, 0))(n_params, out)
        return {"embed": out}

    model = {
        "EdgeFunction": edge_fn,
        "NodeFunction": node_fn,
        "GlobalFunction": None,
    }

    def model_fn(arg): return model[arg]

    final = {
        "EdgeFunction": lambda *x: x[0],
        "NodeFunction": lambda *x: x[0],
        "GlobalFunction": lambda node_attributes, edge_attribtutes, globals_:
        forward_pass(g_params, node_attributes["embed"].reshape(-1)),
        # "GlobalFunction": lambda node_attributes, edge_attribtutes, globals_:
        #     node_attributes["embed"].sum()
    }

    def final_fn(arg): return final[arg]

    net = GraphEncodeNet(mpass, embedding_fn, model_fn, final_fn)
    
    graph = net(graph)

    return graph


def cal_energy(params, graph, **kwargs):
    graph = cal(params, graph, **kwargs)
    return graph.globals.sum()


def cal_acceleration(params, graph, **kwargs):
    graph = cal(params, graph, **kwargs)
    acc_params = params["acc_params"]
    out = jax.vmap(forward_pass, in_axes=(None, 0))(acc_params, graph.nodes["embed"])
    return out

def acceleration_node(R,V, params, **kwargs):
    inp = jnp.hstack([R.flatten(),V.flatten()])
    out = jax.vmap(forward_pass, in_axes=(None, 0))(params, inp)
    return out



In [6]:
# state_graph = jraph.GraphsTuple(nodes={
#     "position": R,
#     "type": species
# },
#     edges={"dij": dij},
#     senders=senders,
#     receivers=receivers,
#     n_node=jnp.array([R.shape[0]]),
#     n_edge=jnp.array([senders.shape[0]]),
#     globals={})


In [7]:
# graph = cal(params['F_pos'], state_graph)

In [8]:
################################################
################### ML Model ###################
################################################
# print("Creating Chain")
x, _, senders, receivers = chain(N)

hidden_dim = [16, 16]
edgesize = 1 # |dij|
nodesize = 2+1 # 2pos,1type
ee = 8
ne = 8
Fparams = dict(
    ee_params=initialize_mlp([edgesize, ee], key),
    ne_params=initialize_mlp([nodesize, ne], key),
    e_params=initialize_mlp([ee+2*ne, *hidden_dim, ee], key),
    n_params=initialize_mlp([2*ee+ne, *hidden_dim, ne], key),
    g_params=initialize_mlp([ne, *hidden_dim, 1], key),
    acc_params=initialize_mlp([ne, *hidden_dim, dim], key),
)
params = {"F_pos": Fparams}


In [9]:

params["gamma"] = initialize_mlp_gamma([1,10,5,1], key)

def nngamma(type, params):
    return forward_pass_gamma(params, type, activation_fn=models.SquarePlus)

def gamma(type, params):
    return vmap(nngamma, in_axes=(0, None))(type.reshape(-1), params).reshape(-1, 1)
    # return nngamma(type.reshape(-1), params["gamma"])#.reshape(-1, 1)

ss = gamma(jax.nn.one_hot(species, 1),params["gamma"])
# gamma(species,params)
ss

Array([[5.2441654],
       [5.2441654],
       [5.2441654],
       [5.2441654],
       [5.2441654]], dtype=float32)

In [10]:
R = Rs[0][0]

def dist(*args):
    disp = displacement(*args)
    return jnp.sqrt(jnp.square(disp).sum())

dij = vmap(dist, in_axes=(0, 0))(R[senders], R[receivers])

def acceleration_fn(params, graph):
    acc = cal_acceleration(params, graph, mpass=1)
    return acc


def acc_fn(species):
    state_graph = jraph.GraphsTuple(nodes={
        "position": R,
        "type": species
    },
        edges={"dij": dij},
        senders=senders,
        receivers=receivers,
        n_node=jnp.array([R.shape[0]]),
        n_edge=jnp.array([senders.shape[0]]),
        globals={})
    
    def apply(R, params):
        state_graph.nodes.update(position=R)
        state_graph.edges.update(dij=vmap(dist, in_axes=(0, 0))(R[senders], R[receivers]))
        return acceleration_fn(params, state_graph)
    return apply

def gamma_fn(species):    
    def fn(params):
        return gamma(jax.nn.one_hot(species, 1),params)    
    return fn

apply_fn = acc_fn(species)
gamma_fn = gamma_fn(species)

def force_fn_model(x, params): return apply_fn(x, params["F_pos"])
def gamma_fn_model(params): return gamma_fn(params["gamma"])

# gamma_fn_model(params)

In [11]:
def next_step_pos_gamma(force_fn_model, gamma_fn_model, shift, dt, kT, mass, runs, key):
    key, split = random.split(key)
    def fn(x, params):
        for i in range(runs):
            # calculate the force
            force = force_fn_model(x, params)
            _gamma = gamma_fn_model(params)
            xi = random.normal(split, x.shape, x.dtype)
            nu = f32(1) / lax.mul(mass.reshape(-1,1) , _gamma)
            x = x+ force * dt * nu + jnp.sqrt(f32(2) * kT * dt * nu) * xi
        return x, _gamma
    return fn


In [12]:
rng_key = random.PRNGKey(0)
rng_key, subkey = random.split(rng_key)

next_step_pos_gamma_fn = next_step_pos_gamma(force_fn_model, gamma_fn_model, shift, dt, kT, masses, runs, subkey)
v_next_step_pos_gamma_fn = vmap(next_step_pos_gamma_fn, in_axes=(0, None))
v_v_next_step_pos_gamma_fn = vmap(v_next_step_pos_gamma_fn, in_axes=(0, None))



In [13]:
@jit
def loss_fn(params, Rs, Rs_1_ac,A=1, B=500): # A=4, B=996 wf=0.996):
    Rs_1_pred, gamma = v_v_next_step_pos_gamma_fn(Rs, params)
    var =1/gamma
    return GaussianNLL(var, Rs_1_pred, Rs_1_ac, A, B)

def gloss(*args):
    return value_and_grad(loss_fn)(*args)

def update(i, opt_state, params, loss__, *data):
    """ Compute the gradient for a batch and update the parameters """
    value, grads_ = gloss(params, *data)
    opt_state = opt_update(i, grads_, opt_state)
    return opt_state, get_params(opt_state), value

@jit
def step(i, ps, *args):
    return update(i, *ps, *args)

opt_init, opt_update_, get_params = optimizers.adam(lr)

@jit
def opt_update(i, grads_, opt_state):
    grads_ = jax.tree_map(jnp.nan_to_num, grads_)
    grads_ = jax.tree_map(partial(jnp.clip, a_min=-1000.0, a_max=1000.0), grads_)
    return opt_update_(i, grads_, opt_state)

def batching(*args, size=None):
    L = len(args[0])
    if size != None:
        nbatches1 = int((L - 0.5) // size) + 1
        nbatches2 = max(1, nbatches1 - 1)
        size1 = int(L/nbatches1)
        size2 = int(L/nbatches2)
        if size1*nbatches1 > size2*nbatches2:
            size = size1
            nbatches = nbatches1
        else:
            size = size2
            nbatches = nbatches2
    else:
        nbatches = 1
        size = L
    
    newargs = []
    for arg in args:
        newargs += [jnp.array([arg[i*size:(i+1)*size]
                                for i in range(nbatches)])]
    return newargs

bRs_in, bRs_out = batching(Rs_in, Rs_out, size=min(len(Rs_in), batch_size))

print(f"training ...")

opt_state = opt_init(params)
epoch = 0
optimizer_step = -1
larray = []
ltarray = []
last_loss = 1000

for epoch in range(epochs):
    l = 0.0
    count = 0
    for data in zip(bRs_in, bRs_out):
        optimizer_step += 1
        opt_state, params, l_ = step(
            optimizer_step, (opt_state, params, 0), *data)
        l += l_
        count+=1
    # print("epoch,countttttt: ", epoch,count)
    # opt_state, params, l_ = step(optimizer_step, (opt_state, params, 0), Rs, Vs, Fs)
    l = l/count
    larray += [l]
    # ltarray += [loss_fn(params, bRs_in, bVs_in, bRs_out)]
    if epoch % 10 == 0:
        print(f"Epoch: {epoch}/{epochs} Loss (MSE):  train={larray[-1]}")#, test={ltarray[-1]}")
    if epoch % 100 == 0:
        print('gammaaaaa: ', gamma_fn_model(params))
        metadata = {
            "savedat": epoch,
            # "mpass": mpass,
            }
        savefile(f"fgnode_trained_model.dil",
                    params, metadata=metadata)
        # savefile(f"loss_array.dil", (larray, ltarray), metadata=metadata)
        savefile(f"loss_array.dil", larray, metadata=metadata)
        if last_loss > larray[-1]:
            last_loss = larray[-1]
            savefile(f"fgnode_trained_model_low.dil",
                        params, metadata=metadata)
        fig, axs = panel(1, 1)
        # plt.semilogy(larray, label="Training")
        plt.plot(larray, label="Training")
        # plt.semilogy(ltarray, label="Test")
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        plt.legend()
        plt.savefig(_filename(f"training_loss.png"))

fig, axs = panel(1, 1)
# plt.semilogy(larray, label="Training")
plt.plot(larray, label="Training")
# plt.semilogy(ltarray, label="Test")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.savefig(_filename(f"training_loss.png"))

params = get_params(opt_state)
savefile(f"fgnode_trained_model.dil", params, metadata=metadata)
# savefile(f"loss_array.dil", (larray, ltarray), metadata=metadata)

if last_loss > larray[-1]:
    last_loss = larray[-1]
    savefile(f"fgnode_trained_model_low.dil", params, metadata=metadata)


training ...
Epoch: 0/10000 Loss (MSE):  train=4.630066394805908
gammaaaaa:  [[5.229261]
 [5.229261]
 [5.229261]
 [5.229261]
 [5.229261]]
=== ../results/a-5-non-linear-Spring-data-brownian_EM-3BFGN/05-18-2023_00-22-54/fgnode_trained_model.dil ===
=== ../results/a-5-non-linear-Spring-data-brownian_EM-3BFGN/05-18-2023_00-22-54/loss_array.dil ===
=== ../results/a-5-non-linear-Spring-data-brownian_EM-3BFGN/05-18-2023_00-22-54/fgnode_trained_model_low.dil ===
=== ../results/a-5-non-linear-Spring-data-brownian_EM-3BFGN/05-18-2023_00-22-54/training_loss.png ===
Epoch: 10/10000 Loss (MSE):  train=4.48037052154541
Epoch: 20/10000 Loss (MSE):  train=4.334883213043213
Epoch: 30/10000 Loss (MSE):  train=4.186748504638672
Epoch: 40/10000 Loss (MSE):  train=4.031232833862305
Epoch: 50/10000 Loss (MSE):  train=3.867236852645874
Epoch: 60/10000 Loss (MSE):  train=3.6995043754577637
Epoch: 70/10000 Loss (MSE):  train=3.5373899936676025
Epoch: 80/10000 Loss (MSE):  train=3.3969926834106445
Epoch: 90/100

/Users/sureshjyoti/GitHub/BGNODE_scratch/.venv_jaxbrow/lib/python3.9/site-packages/shadow/plot.py:181: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure(figsize=figsize, **kwargs)


Epoch: 2010/10000 Loss (MSE):  train=1.5616413354873657
Epoch: 2020/10000 Loss (MSE):  train=1.5614538192749023
Epoch: 2030/10000 Loss (MSE):  train=1.561267614364624
Epoch: 2040/10000 Loss (MSE):  train=1.5610830783843994
Epoch: 2050/10000 Loss (MSE):  train=1.560901165008545
Epoch: 2060/10000 Loss (MSE):  train=1.560720682144165
Epoch: 2070/10000 Loss (MSE):  train=1.5605415105819702
Epoch: 2080/10000 Loss (MSE):  train=1.560364007949829
Epoch: 2090/10000 Loss (MSE):  train=1.5601863861083984
Epoch: 2100/10000 Loss (MSE):  train=1.5600109100341797
gammaaaaa:  [[0.9979802]
 [0.9979802]
 [0.9979802]
 [0.9979802]
 [0.9979802]]
=== ../results/a-5-non-linear-Spring-data-brownian_EM-3BFGN/05-18-2023_00-22-54/fgnode_trained_model.dil ===
=== ../results/a-5-non-linear-Spring-data-brownian_EM-3BFGN/05-18-2023_00-22-54/loss_array.dil ===
=== ../results/a-5-non-linear-Spring-data-brownian_EM-3BFGN/05-18-2023_00-22-54/fgnode_trained_model_low.dil ===
=== ../results/a-5-non-linear-Spring-data-bro

KeyboardInterrupt: 

In [13]:
# params, _ = loadfile(f"fgnode_trained_model_low.dil", verbose=True)

In [13]:
rname=False

# PSYS = f"a-{N}-Spring-data-brownian_EM"
# TAG = f"3BFGN"
# out_dir = f"../results"

def _filename(name, tag=TAG):
    rstring = randfilename if (rname and (tag != "data")) else (
        "0" if (tag == "data") or (withdata == None) else f"0_{withdata}")
    filename_prefix = f"{out_dir}/{PSYS}-{tag}/{rstring}/"
    file = f"{filename_prefix}/{name}"
    os.makedirs(os.path.dirname(file), exist_ok=True)
    filename = f"{filename_prefix}/{name}".replace("//", "/")
    print("===", filename, "===")
    return filename

def OUT(f):
    @wraps(f)
    def func(file, *args, tag=TAG, **kwargs):
        return f(_filename(file, tag=tag), *args, **kwargs)
    return func

loadmodel = OUT(src.models.loadmodel)
savemodel = OUT(src.models.savemodel)

loadfile = OUT(src.io.loadfile)
savefile = OUT(src.io.savefile)
save_ovito = OUT(src.io.save_ovito)


In [15]:
params, _ = loadfile(f"fgnode_trained_model_low.dil", verbose=True)

=== ../results/a-5-Spring-data-brownian_EM-3BFGN/0/fgnode_trained_model_low.dil ===
Loading ../results/a-5-Spring-data-brownian_EM-3BFGN/0/fgnode_trained_model_low.dil


In [14]:
gamma_fn_model(params)

Array([[0.9980434],
       [0.9980434],
       [0.9980434],
       [0.9980434],
       [0.9980434]], dtype=float32)

In [15]:
# %matplotlib inline
# import matplotlib.pyplot as plt

spring_constant = 1.0
length_constant = 1.0
gamma_orig = jnp.ones(jnp.unique(species).shape)
stride = 1
runs=100

def SPRING(x, stiffness=1.0, length=1.0):
    x_ = jnp.linalg.norm(x, keepdims=True)
    return 0.5*stiffness*(x_ - length)**4

def pot_energy_orig(x):
    dr = x[senders, :] - x[receivers, :]
    return vmap(partial(SPRING, stiffness=spring_constant, length=length_constant))(dr).sum()

def force_fn_orig(R, params):
    return -grad(pot_energy_orig)(R)


def get_forward_sim(params = None, force_fn = None, gamma = None, runs=10):
        @jit
        def fn(R,key):
            return predition_brow(R, params, force_fn, shift, dt, kT, masses, gamma = gamma, stride=stride, runs=runs, key=key)
        return fn

gamma_model = gamma_fn_model(params)

sim_orig = get_forward_sim(params=None,force_fn=force_fn_orig, gamma=gamma_orig,runs=runs)
sim_model = get_forward_sim(params=params,force_fn=force_fn_model, gamma=gamma_model,runs=runs)



In [16]:
plotthings = True
rng_key = random.PRNGKey(0)
maxtraj = 100

_gamma = gamma_fn_model(params)
nexp = {
        "dz_actual": [],
        "dz_pred": [],
        "z_actual": [],
        "z_pred": [],
        "_gamma": [_gamma]
        }

trajectories = []
for ind in range(maxtraj):
    print(f"Simulating trajectory {ind}/{maxtraj} ...")
    R, _ = chain(N)[:2]
    for rand in range(10):
        rng_key, subkey = random.split(rng_key)
        actual_traj = sim_orig(R,(ind+13)*subkey)
        rng_key, subkey = random.split(rng_key)
        pred_traj = sim_model(R, (ind+13)*subkey)
        
        nexp["dz_actual"] += [actual_traj.position-R]
        nexp["dz_pred"] += [pred_traj.position-R]
        
        nexp["z_actual"] += [actual_traj.position]
        nexp["z_pred"] += [pred_traj.position]
        
        if save_ovito:
            if ind<2 and rand<5:
                save_ovito(f"actual_{ind}_{rand}.xyz", [state for state in BrownianStates(actual_traj)], lattice="")
                save_ovito(f"pred_{ind}_{rand}.xyz", [state for state in BrownianStates(pred_traj)], lattice="")
        
        trajectories += [(actual_traj, pred_traj)]
        if ind%10==0:
            savefile("trajectories.pkl", trajectories)

savefile(f"error_parameter.pkl", nexp)
savefile("trajectories.pkl", trajectories)


Simulating trajectory 0/100 ...
=== ../results/a-5-Spring-data-brownian_EM-3BFGN/0/actual_0_0.xyz ===
Saving ovito file: ../results/a-5-Spring-data-brownian_EM-3BFGN/0/actual_0_0.xyz
=== ../results/a-5-Spring-data-brownian_EM-3BFGN/0/pred_0_0.xyz ===
Saving ovito file: ../results/a-5-Spring-data-brownian_EM-3BFGN/0/pred_0_0.xyz
=== ../results/a-5-Spring-data-brownian_EM-3BFGN/0/trajectories.pkl ===
=== ../results/a-5-Spring-data-brownian_EM-3BFGN/0/actual_0_1.xyz ===
Saving ovito file: ../results/a-5-Spring-data-brownian_EM-3BFGN/0/actual_0_1.xyz
=== ../results/a-5-Spring-data-brownian_EM-3BFGN/0/pred_0_1.xyz ===
Saving ovito file: ../results/a-5-Spring-data-brownian_EM-3BFGN/0/pred_0_1.xyz
=== ../results/a-5-Spring-data-brownian_EM-3BFGN/0/trajectories.pkl ===
=== ../results/a-5-Spring-data-brownian_EM-3BFGN/0/actual_0_2.xyz ===
Saving ovito file: ../results/a-5-Spring-data-brownian_EM-3BFGN/0/actual_0_2.xyz
=== ../results/a-5-Spring-data-brownian_EM-3BFGN/0/pred_0_2.xyz ===
Saving ov